In [1]:
# import necessary libraries
import sagemaker
from sagemaker.xgboost.estimator import XGBoost
from sagemaker.session import Session
from sagemaker.inputs import TrainingInput
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner
)
from sagemaker import get_execution_role
import boto3

role = get_execution_role()
default_bucket = "sagemaker-data113"

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[02/03/25 15:56:37] INFO     Found credentials from IAM Role:                                   ]8;id=56958;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=148304;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


                    INFO     Found credentials from IAM Role:                                   ]8;id=562158;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=201600;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

## XGBoost Hyper-paramter Tuning and Training jobs

In [2]:
# Creating Training and Validation data channels from s3 buckets (saved in 'churn_data_prep.ipynb')
s3_input_train = TrainingInput(s3_data=f"s3://{default_bucket}/train.csv", content_type="csv")
s3_input_validation = TrainingInput(s3_data=f"s3://{default_bucket}/validation.csv", content_type="csv")

Following hyper-parameters are fixed 

- `metric` (default value for binary classification) error rate  = #(wrong_cases)/#(total_cases) at threshold of 0.5.
- `objective`  logistic regression for binary classification, output probability 
- `num_round` controls the number of boosting rounds. This is essentially the subsequent models that are trained using the residuals of previous iterations. Again, more rounds should produce a better fit on the training data, but can be computationally expensive or lead to overfitting.
- `rate_drop` The dropout rate that specifies the fraction of previous trees to drop during the dropout.



In [3]:
fixed_hyperparameters = {
    "eval_metric": "auc",
    "objective":"binary:logistic",
    "num_round":"100",
    "rate_drop":"0.3",
}

In [4]:
sess = sagemaker.Session()
container = sagemaker.image_uris.retrieve("xgboost", sess.boto_region_name, "1.5-1")

estimator = sagemaker.estimator.Estimator(
    container,
    role,
    instance_count=1,
    hyperparameters=fixed_hyperparameters,
    instance_type="ml.m4.xlarge",
    output_path="s3://{}/output".format(default_bucket),
    sagemaker_session=sess
)

[02/03/25 15:56:38] INFO     Found credentials from IAM Role:                                   ]8;id=700105;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=765857;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

                    INFO     Ignoring unnecessary instance type: None.                            ]8;id=123295;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=738134;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#528\528]8;;\

                    INFO     Found credentials from IAM Role:                                   ]8;id=60518;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=832730;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

Following hyperparamters are varied for tuning:-

- `eta` controls how aggressive each round of boosting is. Larger values lead to more conservative boosting.
- `min_child_weight` Minimum sum of instance weight (hessian) needed in a child. If the tree partition step results in a leaf node with the sum of instance weight less than min_child_weight, the building process gives up further partitioning. The larger the tree, the more conservative it is.
- `alpha` L1 regularization term on weights. Increasing this value makes models more conservative.
- `max_depth` Maximum depth of a tree. Increasing this value makes the model more complex and likely to be overfit.

In [5]:
hyperparameter_ranges = {
    "eta": ContinuousParameter(0, 1),
    "min_child_weight": ContinuousParameter(1, 10),
    "alpha": ContinuousParameter(0, 2),
    "max_depth": IntegerParameter(1, 10),
}

In [6]:
objective_metric_name = "validation:auc"

In [7]:
tuner = HyperparameterTuner(
    estimator,objective_metric_name,hyperparameter_ranges,max_jobs=10,max_parallel_jobs=2)

In [8]:
tuner.fit({
    "train":s3_input_train,
    "validation":s3_input_validation
    },include_cls_metadata=False)

                    WARNING  No finished training job found associated with this estimator.       ]8;id=213142;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/estimator.py\estimator.py]8;;\:]8;id=202408;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/estimator.py#1914\1914]8;;\
                             Please make sure this estimator is only used for building workflow                    
                             config                                                                                

                    WARNING  No finished training job found associated with this estimator.       ]8;id=880526;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/estimator.py\estimator.py]8;;\:]8;id=47579;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/estimator.py#1914\1914]8;;\
                             Please make sure this estimator is only used for building workflow                    
                             config                                                                                

                    INFO     Creating hyperparameter tuning job with name:                          ]8;id=599556;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=676928;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#3383\3383]8;;\
                             sagemaker-xgboost-250203-1556                                                         

................................................................................!


In [9]:
tuning_job_result = boto3.client("sagemaker").describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuner.latest_tuning_job.job_name
)
job_count = tuning_job_result["TrainingJobStatusCounters"]["Completed"]
print("%d training jobs have completed" %job_count)

10 training jobs have completed


### Fetch Tuning results

In [14]:
import pandas as pd
tuner = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_result["HyperParameterTuningJobName"])

full_df = tuner.dataframe()

objective = tuning_job_result["HyperParameterTuningJobConfig"]["HyperParameterTuningJobObjective"]
is_minimize = objective["Type"] != "Maximize"
objective_name = objective["MetricName"]

if len(full_df) > 0:
    df = full_df[full_df["FinalObjectiveValue"] > -float("inf")]
    if len(df) > 0:
        df = df.sort_values("FinalObjectiveValue", ascending=is_minimize)
        print("Number of training jobs with valid objective: %d" % len(df))
        print({"lowest": min(df["FinalObjectiveValue"]), "highest": max(df["FinalObjectiveValue"])})
        pd.set_option("display.max_colwidth", None)  # Don't truncate TrainingJobName
    else:
        print("No training jobs have reported valid results yet.")

df

Number of training jobs with valid objective: 10
{'lowest': 0.5, 'highest': 0.8387399911880493}


,alpha,eta,max_depth,min_child_weight,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
8,0.733755,0.021616,9.0,1.983949,sagemaker-xgboost-250203-1556-002-fa2305d5,Completed,0.83874,2025-02-03 15:57:31+00:00,2025-02-03 15:59:30+00:00,119.0
6,1.689340,0.298164,1.0,7.043998,sagemaker-xgboost-250203-1556-004-df487b17,Completed,0.83788,2025-02-03 15:59:40+00:00,2025-02-03 16:00:09+00:00,29.0
0,0.934953,0.094159,2.0,4.478499,sagemaker-xgboost-250203-1556-010-5f9a43be,Completed,0.83607,2025-02-03 16:02:50+00:00,2025-02-03 16:03:19+00:00,29.0
5,1.475549,0.669232,1.0,2.908051,sagemaker-xgboost-250203-1556-005-9baaf75c,Completed,0.83571,2025-02-03 16:00:23+00:00,2025-02-03 16:00:57+00:00,34.0
7,1.570456,0.257443,6.0,2.324659,sagemaker-xgboost-250203-1556-003-6cf75fc0,Completed,0.81876,2025-02-03 15:59:38+00:00,2025-02-03 16:00:11+00:00,33.0
1,0.907866,0.324611,10.0,5.619041,sagemaker-xgboost-250203-1556-009-4a970bbb,Completed,0.80885,2025-02-03 16:02:20+00:00,2025-02-03 16:02:49+00:00,29.0
9,1.366393,0.931646,10.0,7.079089,sagemaker-xgboost-250203-1556-001-45893fcb,Completed,0.79515,2025-02-03 15:57:21+00:00,2025-02-03 15:59:20+00:00,119.0
4,1.681018,0.931486,9.0,8.379898,sagemaker-xgboost-250203-1556-006-e6bc5fb8,Completed,0.78780,2025-02-03 16:00:29+00:00,2025-02-03 16:01:03+00:00,34.0
2,0.292260,0.000000,5.0,1.208380,sagemaker-xgboost-250203-1556-008-72ff8b26,Completed,0.50000,2025-02-03 16:01:30+00:00,2025-02-03 16:02:01+00:00,31.0
3,0.000000,0.000000,10.0,2.748000,sagemaker-xgboost-250203-1556-007-e68f217d,Completed,0.50000,2025-02-03 16:01:26+00:00,2025-02-03 16:02:02+00:00,36.0


In [15]:
best_hyperparameters = tuning_job_result["BestTrainingJob"]["TunedHyperParameters"]
best_hyperparameters

{'alpha': '0.7337550802445072',
 'eta': '0.02161603441293436',
 'max_depth': '9',
 'min_child_weight': '1.983949242105516'}

The scatter plot shows that the points are distributed quite apart from each other. Hence, we have set the ranges well for hyperparamter optimization.


## Register the model

Go to Amazon Sagemaker console. In the Training -> Hyperparameter Tuning Jobs, select the hyperparamter tuning job with the corresponding name ,initiated in this notebook (or use Creation Time column).  There, in the hyperparamter tuning job, there will be tab showing the Best Trained Model summary and a button "Create Model", to  register the model container. We will later use this image to create a deployment endpoint for real-time prediction.